In [1]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import zz_feature_map
from qiskit_aer.primitives import SamplerV2
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.state_fidelities import ComputeUncompute
import pandas as pd
import numpy as np

In [2]:
from qiskit_aer.primitives import SamplerV2 as Sampler

backend_options = {        
    'device': "GPU",
    }

options = {
    'backend_options': backend_options
    }

sampler = Sampler(options = options)

In [4]:
feature_dimension=8
reps=2

# Data paths
TRAIN_FILE = '../Data/X_train_scaled.csv'
TEST_FILE = '../Data/X_test_scaled.csv'
LABEL_TRAIN_FILE = '../2025-Quantathon-Tornado-Q-training_data-640-examples.xlsx'
LABEL_TEST_FILE = '../2025-Quantum-Tornado-Q-test_data-200-examples.xlsx'

# Load training data
df_train = pd.read_csv(TRAIN_FILE)
# Load test data
df_test = pd.read_csv(TEST_FILE)

df_label_train_binary = pd.read_excel(LABEL_TRAIN_FILE)["ef_binary"]
df_label_train_class = pd.read_excel(LABEL_TRAIN_FILE)["ef_class"]

df_test_train_binary = pd.read_excel(LABEL_TEST_FILE)["ef_binary"]
df_test_train_class = pd.read_excel(LABEL_TEST_FILE)["ef_class"]

print(f"✓ Training data loaded: {df_train.shape[0]} rows, {df_train.shape[1]} columns")
print(f"✓ Test data loaded: {df_test.shape[0]} rows, {df_test.shape[1]} columns")


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
# ------------------------------------------------------------
# Public entry point: build a circuit that (1) encodes features
# and (2) applies a configurable entangling block.
# ------------------------------------------------------------
def build_circuit(
    data: Iterable[float],
    *,
    # ---- Encoding controls ----
    normalize: bool = True,
    angle_scale: float = np.pi,                  # multiply angles by this (e.g., π)
    encoding_axes: Sequence[str] = ("rx", "ry"), # which rotations to use per feature
                                                 # e.g., ("ry",) or ("ry","rz") etc.
    # ---- Entangler controls ----
    entanglement: Literal["full", "ring", "linear"] = "full",
    gate:        Literal["cx", "cz"] = "cx",
    num_layers:  int = 2,
    alternate_directions: bool = True,
    add_barriers: bool = True,
) -> QuantumCircuit:
    """
    Builds a feature-generating circuit for your shadows pipeline:
      data -> [angle encoding] -> [entangling layers]

    Parameters
    ----------
    data : Iterable[float]
        Your feature vector (one qubit per feature).
    normalize : bool
        Whether to apply tanh normalization (maps to [-1,1]) before angle scaling.
        Recommended for robustness.
    angle_scale : float
        Scalar to multiply features when used as rotation angles (default π).
    encoding_axes : Sequence[str]
        Rotations to apply per feature, in order. Options: "rx", "ry", "rz".
        Example: ("ry",) or ("ry","rz") or ("rx","ry","rz").
    entanglement : {"full","ring","linear"}
        Connectivity of the entangling block.
    gate : {"cx","cz"}
        Two-qubit gate family. Use "cz" if your backend favors symmetric CZ.
    num_layers : int
        Number of repeated entangling layers.
    alternate_directions : bool
        If using CX, flip control/target each layer to reduce directional bias.
    add_barriers : bool
        Add visual/compile barriers between layers (useful during debugging).

    Returns
    -------
    QuantumCircuit
        Circuit with encoding + entanglement applied.
    """
    data = np.asarray(list(data), dtype=float)
    n = int(data.size)
    assert n >= 2, "Need at least 2 qubits (features) to add entanglement."

    # ---------- 1) Normalize & scale angles ----------
    # Good default for continuous geophysical features (robust to outliers).
    if normalize:
        data = np.tanh(data)               # maps to [-1,1]
    thetas = angle_scale * data            # rescale to angles

    # ---------- 2) Encoding ----------
    qc = QuantumCircuit(n, name="encode+entangle")
    _apply_angle_encoding(qc, thetas, encoding_axes)

    # ---------- 3) Entangling block ----------
    _add_entangling_layer(
        qc,
        num_layers=num_layers,
        entanglement=entanglement,
        gate=gate,
        alternate_directions=alternate_directions,
        add_barriers=add_barriers,
    )
    return qc


In [ ]:
#Renormalize Data

df_train = np.tanh(df_train)
df_test = np.tanh(df_test)

In [ ]:
#Data Encoding (zz)
feature_map = zz_feature_map(feature_dimension=feature_dimension, reps=reps, entanglement="linear")

#Why this kind of data encoding?

In [ ]:
#Create Quantum Kernel
quantum_kernel = FidelityQuantumKernel(feature_map=feature_map, fidelity=ComputeUncompute(sampler))

In [ ]:
#Train the QSVC

qsvc = QSVC(quantum_kernel=quantum_kernel)
qsvc.fit(df_train, df_label_train_binary)

kernel_matrix_train = quantum_kernel.evaluate(df_train)  # shape: (n_samples, n_samples)
kernel_matrix_test = quantum_kernel.evaluate(df_test, df_train)  # shape: (n_test, n_train)

In [ ]:
#Analyze Results
import matplotlib.pyplot as plt

plt.imshow(kernel_matrix_train, cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.title("Kernel Matrix")
plt.show()

plt.imshow(kernel_matrix_test, cmap='Blues', interpolation='nearest')
plt.colorbar()
plt.title("Test Matrix")
plt.show()

score = qsvc.score(test_features, test_labels)
print(score)

In [ ]:
#Measure Kernel Alignment
#
#measure of how well your kernel separates classes
#

In [ ]:
#Cross-Validation with QSVC or Classical SVC
#
#
#

In [ ]:
#Compare with Classical Kernels
#
#
#

In [ ]:
#Kernel Spectrum / Eigenvalues
#
#
#

In [ ]:
#Sensitivity Analysis
#
#
#